In [1]:
import torch

In [ ]:
class MultiLayerPerception(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax = False):
        intermediate = torch.nn.